In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip
!pip install implicit

In [ ]:
import random
import math
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from copy import deepcopy
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
from collections import OrderedDict
from tqdm import tqdm
import torch.nn.functional as F
import implicit

https://github.com/microsoft/recommenders/blob/main/examples/06_benchmarks/movielens.ipynb

* Use SVD for predicting explicit data (e.g. ratings such as those in the Netflix Prize)

* Use ALS for predicting implicit data (e.g. played/not played or Personalized Ranking)

* Use NMF to perform clustering and then use resulting clusters to generate features (e.g. as in "Non-negative matrix factorization for semi-supervised data clustering")

# 1) Neural Collaborative Filtering (NCF)

**Matrix Factorization** has its limitation to reflect the interactions between user and items. Neural Collaborative Filtering as shown in the graph below. 

<img src="https://miro.medium.com/max/828/1*sTBtqrsQzTKlZ8hSU7I6FQ.webp">

In the input layer, the user and item are one-hot encoded. Then, they are mapped to the hidden space with embedding layers accordingly. The Neural FC layer can be any kind neuron connections. Multiple layer perceptron, for example, can be placed here. It claims that with the complicated connection and non-linearity in the Neural CF layers, this model is capable of properly estimating the complex interactions between user and item in the latent space.

<img src="https://miro.medium.com/max/828/1*CoETyuU36fshduKAfFhCrg.webp">

In order to introduce additional non-linearity, the final model proposed, NeuMF, includes a Mutliple-layer Perceptron (MLP) module apart from the Generalized Matrix Factorization (GMP) layer. The output of GMF and MLP modules are concatenated and connected with the sigmoid activation output layer.

## 1.1 Data Loading

In [ ]:
colnames = ['user_id', 'movie_id', 'rating', 'time']
df_train = pd.read_csv('ml-100k/u1.base', delimiter='\t', header=None, names=colnames)
df_test = pd.read_csv('ml-100k/u1.test', delimiter='\t', header=None, names=colnames)
df_full = pd.concat([df_train, df_test])
df_full.head()

,user_id,movie_id,rating,time
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [ ]:
### processing
def binarize(ratings):
    """binarize into 0 or 1, imlicit feedback"""
    ratings = deepcopy(ratings)
    # ratings['rating'][ratings['rating'] > 0] = 1.0
    ratings.loc[ratings['rating'] > 0, 'rating'] = 1.0

    return ratings

def sample_negative(ratings, item_pool):
    """return sampled 100 negative items"""

    interact_status = ratings.groupby('user_id')['movie_id'].apply(set).reset_index().rename(
        columns={'movie_id': 'interacted_items'})
        
    interact_status['negative_items'] = interact_status['interacted_items'].apply(
        lambda x: item_pool - x)
    
    interact_status['negative_samples'] = interact_status['negative_items'].apply(
        lambda x: random.sample(x, 99))
    
    return interact_status[['user_id', 'negative_items', 'negative_samples']]

def split_loo(ratings):
    """train/test split using leave one out strategy"""

    ratings['rank_latest'] = ratings.groupby(['user_id'])['time'].rank(
        method='first', ascending=False)
    
    test = ratings[ratings['rank_latest'] == 1]
    train = ratings[ratings['rank_latest'] > 1]

    assert train['user_id'].nunique() == test['user_id'].nunique()
    return train[['user_id', 'movie_id', 'rating']], test[['user_id', 'movie_id', 'rating']]

class UserItemRatingDataset(Dataset):
    """Wrapper, convert <user, item, rating> Tensor into Pytorch Dataset"""
    
    def __init__(self, user_tensor, item_tensor, target_tensor):
        self.user_tensor = user_tensor
        self.item_tensor = item_tensor
        self.target_tensor = target_tensor

    def __getitem__(self, index):
        return self.user_tensor[index], self.item_tensor[index], self.target_tensor[index]

    def __len__(self):
        return self.user_tensor.size(0)

# explicit feedback to implicit
df_process = binarize(df_full)

user_pool = set(df_process['user_id'].unique())
item_pool = set(df_process['movie_id'].unique())
df_negative = sample_negative(df_full, item_pool)
df_train, df_test = split_loo(df_process)

# add negative data for implicit data (no rating data)
num_neg = 2
df_train = pd.merge(df_train, df_negative[['user_id', 'negative_items']], on='user_id')
df_train['negatives'] = df_train['negative_items'].apply(lambda x: random.sample(x, num_neg))
df_test = pd.merge(df_test, df_negative[['user_id', 'negative_items']], on='user_id')
df_test['negatives'] = df_test['negative_items'].apply(lambda x: random.sample(x, num_neg))

users_train = df_train['user_id'].values.tolist()
items_train = df_train['movie_id'].values.tolist()
ratings_train = df_train['rating'].astype(float).values.tolist()

for i in range(num_neg):
    users_train += df_train['user_id'].values.tolist()
    items_train += df_train['negatives'].apply(lambda x: x[i]).values.tolist()
    ratings_train += [0] * len(df_train)

train_dataset = UserItemRatingDataset(
    user_tensor=torch.LongTensor(users_train),
    item_tensor=torch.LongTensor(items_train),
    target_tensor=torch.FloatTensor(ratings_train))

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
user_sample, item_sample, target_sample = next(iter(train_loader))
user_sample.size()

torch.Size([256])

In [ ]:
### utils
def resume_checkpoint(model, model_dir, device_id):
    state_dict = torch.load(
        model_dir,
        map_location=lambda storage, loc: storage.cuda(device=device_id)
        ) 
    model.load_state_dict(state_dict)

def save_checkpoint(model, model_dir):
    torch.save(model.state_dict(), model_dir)

def use_optimizer(network, params):
    if params['optimizer'] == 'sgd':
        optimizer = torch.optim.SGD(
            network.parameters(),
            lr=params['sgd_lr'],
            momentum=params['sgd_momentum'],
            weight_decay=params['l2_regularization']
            )
        
    elif params['optimizer'] == 'adam':
        optimizer = torch.optim.Adam(
            network.parameters(), 
            lr=params['adam_lr'],
            weight_decay=params['l2_regularization']
            )
        
    elif params['optimizer'] == 'rmsprop':
        optimizer = torch.optim.RMSprop(
            network.parameters(),
            lr=params['rmsprop_lr'],
            alpha=params['rmsprop_alpha'],
            momentum=params['rmsprop_momentum']
            )
    return optimizer

## 1.2 GMF Layer

In [ ]:
gmf_config = {
    'alias': 'gmf_factor8neg2-implict',
    'num_epoch': 20,
    'batch_size': 256,
    'optimizer': 'adam',
    'adam_lr': 1e-3,
    'num_users': len(user_pool) + 1,
    'num_items': len(item_pool) + 1,
    'latent_dim': 8,
    'num_negative': num_neg,
    'l2_regularization': 0.01,
    'use_cuda': True,
    'device_id': 0,
    'model_dir':'checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model'
}

class GMF(torch.nn.Module):
    def __init__(self, config):
        super(GMF, self).__init__()
        self.num_users = config['num_users']
        self.num_items = config['num_items']
        self.latent_dim = config['latent_dim']

        self.embedding_user = torch.nn.Embedding(
            num_embeddings=self.num_users, embedding_dim=self.latent_dim)
        
        self.embedding_item = torch.nn.Embedding(
            num_embeddings=self.num_items, embedding_dim=self.latent_dim)

        self.affine_output = torch.nn.Linear(
            in_features=self.latent_dim, out_features=1)
        
        self.logistic = torch.nn.Sigmoid()

    def forward(self, user_indices, item_indices):

        # (m, latent_dim)
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        element_product = torch.mul(user_embedding, item_embedding)

        # (m, 1)
        logits = self.affine_output(element_product)
        rating = self.logistic(logits)
        return rating

    def init_weight(self):
        pass

### testing
gmf_model = GMF(gmf_config)
result = gmf_model(user_sample, item_sample)
print(f"Result Size: {result.size()}")

Result Size: torch.Size([256, 1])


## 1.3 MLP Layer

In [ ]:
# fit trained gmf layer into MLP

mlp_config = {
    'alias': 'mlp_factor8neg4_bz256_166432168_pretrain_reg_0.0000001',
    'num_epoch': 20,
    'batch_size': 256,
    'optimizer': 'adam',
    'adam_lr': 1e-3,
    'num_users': len(user_pool) + 1,
    'num_items': len(item_pool) + 1,
    'latent_dim': 8,
    'num_negative': 2,
    'layers': [16, 64, 32, 16, 8],  # layers[0] is the concat of latent user & latent item
    'l2_regularization': 0.0000001,  # MLP model is sensitive to hyper params
    'use_cuda': False,
    'device_id': 7,
    'pretrain': False,
    'model_dir':'checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model',
    'pretrain_mf': True,
    'pretrain_mf_loc': 'checkpoints/{}'.format('gmf_factor8neg4_Epoch100_HR0.6391_NDCG0.2852.model'),
    }

class MLP(torch.nn.Module):
    def __init__(self, config, gmf_model=None):

        super(MLP, self).__init__()
        self.config = config
        self.num_users = config['num_users']
        self.num_items = config['num_items']
        self.latent_dim = config['latent_dim']

        self.embedding_user = torch.nn.Embedding(
            num_embeddings=self.num_users, embedding_dim=self.latent_dim)
        
        self.embedding_item = torch.nn.Embedding(
            num_embeddings=self.num_items, embedding_dim=self.latent_dim)

        self.fc_layers = torch.nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(config['layers'][:-1], config['layers'][1:])):
            self.fc_layers.append(torch.nn.Linear(in_size, out_size))

        self.affine_output = torch.nn.Linear(in_features=config['layers'][-1], out_features=1)
        self.logistic = torch.nn.Sigmoid()

        if config['pretrain_mf']:
            self.load_pretrain_weights(gmf_model)

    def forward(self, user_indices, item_indices):

        # (m, latent_dim)
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)

        # the concat latent vector, (m, latent_dim * 2)
        vector = torch.cat([user_embedding, item_embedding], dim=-1)  

        # (m, latent_dim * 2) --> (m, 64, 32, ..., 8)
        for idx, _ in enumerate(range(len(self.fc_layers))):
            vector = self.fc_layers[idx](vector)
            vector = torch.nn.ReLU()(vector)
            # vector = torch.nn.BatchNorm1d()(vector)
            # vector = torch.nn.Dropout(p=0.5)(vector)

        # (m, 1)
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating

    def init_weight(self):
        pass

    def load_pretrain_weights(self, gmf_model=None):
        """Loading weights from trained GMF model"""

        print("Loading GMF Model Weight")

        if gmf_model is None:
            gmf_model = GMF(self.config)
            resume_checkpoint(
                gmf_model, model_dir=self.config['pretrain_mf_loc'], device_id=self.config['device_id'])

        if self.config['use_cuda'] is True:
            gmf_model.cuda()

        self.embedding_user.weight.data = gmf_model.embedding_user.weight.data
        self.embedding_item.weight.data = gmf_model.embedding_item.weight.data

### testing
mlp_model = MLP(mlp_config, gmf_model)
result = mlp_model(user_sample, item_sample)
print(f"Result Size: {result.size()}")

Loading GMF Model Weight
Result Size: torch.Size([256, 1])


## 1.4 NeuMF Layer

In [ ]:
neumf_config = {
    'alias': 'pretrain_neumf_factor8neg4',
    'num_epoch': 20,
    'batch_size': 256,
    'optimizer': 'adam',
    'adam_lr': 1e-3,
    'num_users': len(user_pool) + 1,
    'num_items': len(item_pool) + 1,
    'latent_dim_mf': 8,
    'latent_dim_mlp': 8,
    'num_negative': 2,
    'layers': [16, 64, 32, 16, 8],
    'l2_regularization': 0.01,
    'use_cuda': False,
    'device_id': 7,
    'model_dir':'checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model',
    'pretrain': True,
    'pretrain_mf_loc': 'checkpoints/{}'.format('gmf_factor8neg4_Epoch100_HR0.6391_NDCG0.2852.model'),
    'pretrain_mlp_loc': 'checkpoints/{}'.format('mlp_factor8neg4_Epoch100_HR0.5606_NDCG0.2463.model'),
    }

class NeuMF(torch.nn.Module):
    def __init__(self, config, gmf_model=None, mlp_model=None):

        super(NeuMF, self).__init__()
        self.config = config
        self.num_users = config['num_users']
        self.num_items = config['num_items']
        self.latent_dim_mf = config['latent_dim_mf']
        self.latent_dim_mlp = config['latent_dim_mlp']

        self.embedding_user_mlp = torch.nn.Embedding(
            num_embeddings=self.num_users, embedding_dim=self.latent_dim_mlp)
        
        self.embedding_item_mlp = torch.nn.Embedding(
            num_embeddings=self.num_items, embedding_dim=self.latent_dim_mlp)
        
        self.embedding_user_mf = torch.nn.Embedding(
            num_embeddings=self.num_users, embedding_dim=self.latent_dim_mf)
        
        self.embedding_item_mf = torch.nn.Embedding(
            num_embeddings=self.num_items, embedding_dim=self.latent_dim_mf)
        
        self.fc_layers = torch.nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(config['layers'][:-1], config['layers'][1:])):
            self.fc_layers.append(torch.nn.Linear(in_size, out_size))

        self.affine_output = torch.nn.Linear(
            in_features=config['layers'][-1] + config['latent_dim_mf'], out_features=1)
        self.logistic = torch.nn.Sigmoid()

        if config['pretrain']:
            self.load_pretrain_weights(gmf_model, mlp_model)

    def forward(self, user_indices, item_indices):

        # (m, latent_dim_mlp)
        user_embedding_mlp = self.embedding_user_mlp(user_indices)
        item_embedding_mlp = self.embedding_item_mlp(item_indices)

        # (m, latent_dim_mf)
        user_embedding_mf = self.embedding_user_mf(user_indices)
        item_embedding_mf = self.embedding_item_mf(item_indices)

        # (m, latent_dim_mlp * 2)
        mlp_vector = torch.cat([user_embedding_mlp, item_embedding_mlp], dim=-1)

        # (m, latent_dim_mf)
        mf_vector = torch.mul(user_embedding_mf, item_embedding_mf)

        # (m, latent_dim_mlp * 2) --> (m, 64) --> (m, latent_dim_mlp)
        for idx, _ in enumerate(range(len(self.fc_layers))):
            mlp_vector = self.fc_layers[idx](mlp_vector)
            mlp_vector = torch.nn.ReLU()(mlp_vector)

        # (m, enumerate + latent_dim_mf)
        vector = torch.cat([mlp_vector, mf_vector], dim=-1)

        # (m, 1)
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating

    def init_weight(self):
        pass

    def load_pretrain_weights(self, gmf_model=None, mlp_model=None):
        """Loading weights from trained MLP model & GMF model"""

        ### MLP
        print("Loading Weight for MLP")
        config = self.config
        config['latent_dim'] = config['latent_dim_mlp']

        if mlp_model is None:
            mlp_model = MLP(config)
            resume_checkpoint(
                mlp_model, model_dir=config['pretrain_mlp'], device_id=config['device_id'])
                    
        self.embedding_user_mlp.weight.data = mlp_model.embedding_user.weight.data
        self.embedding_item_mlp.weight.data = mlp_model.embedding_item.weight.data
        for idx in range(len(self.fc_layers)):
            self.fc_layers[idx].weight.data = mlp_model.fc_layers[idx].weight.data

        ### GMF
        print("Loading Weight for GMF")
        config['latent_dim'] = config['latent_dim_mf']

        if gmf_model is None:
            gmf_model = GMF(config)
            resume_checkpoint(gmf_model, model_dir=config['pretrain_mf'], device_id=config['device_id'])

        if config['use_cuda'] is True:
            mlp_model.cuda()
            gmf_model.cuda()
        
        self.embedding_user_mf.weight.data = gmf_model.embedding_user.weight.data
        self.embedding_item_mf.weight.data = gmf_model.embedding_item.weight.data

        self.affine_output.weight.data = 0.5 * torch.cat(
            [mlp_model.affine_output.weight.data, gmf_model.affine_output.weight.data], dim=-1
            )
        
        self.affine_output.bias.data = 0.5 * \
            (mlp_model.affine_output.bias.data + gmf_model.affine_output.bias.data)

### testing
neumf_model = NeuMF(neumf_config, gmf_model, mlp_model)
pred = neumf_model(user_sample, item_sample)
print(f"Result Size: {pred.size()}")

Loading Weight for MLP
Loading Weight for GMF
Result Size: torch.Size([256, 1])


## 1.5 Evaluation

* **Hit Ratio @ K** denotes the fraction of prediction hits given K recommendation for each user. Let’s say if we recommend 10 items for each user, and 4 of the 10 users interact the items matching our recommendation, then Hit Ratio@ 10 = 0.4. 

* **Normalized Discounted Cumulative Gain (NDCG)** can be viewed as an extension of Hit Ratio, except that it considers the order of the hit. This means that if you the hit occurs at the higher recommendation, NDCG will be higher.

In [ ]:
### loss function
criterion = torch.nn.BCELoss()
loss = criterion(pred.view(-1), target_sample)
print(loss)

tensor(0.7223, grad_fn=<BinaryCrossEntropyBackward0>)


In [ ]:
# for evaluation, give 1 positive sample and N negative sample to each user
# the model will rank the N + 1 items

g_truth = dict(zip(df_test['user_id'], df_test['movie_id']))
d1 = df_test[['user_id', 'negatives']].copy()
d1 = d1.explode('negatives')
d1['rating'] = 0
d1.rename(columns={'negatives' : 'movie_id'}, inplace=True)
df_eval = pd.concat([d1, df_test[['user_id', 'movie_id', 'rating']]])
df_eval.head()

,user_id,movie_id,rating
0,3,1262,0
0,3,832,0
1,4,1011,0
1,4,928,0
2,5,1488,0


In [ ]:
### rank according to score
pred = neumf_model(
    torch.LongTensor(df_eval['user_id'].values.tolist()), 
    torch.LongTensor(df_eval['movie_id'].values.tolist())
    )
df_eval['score'] = pred.view(-1).detach().numpy()
df_eval['rank'] = df_eval.groupby('user_id')['score'].rank(method='first', ascending=False)
df_eval.sort_values(['user_id', 'rank'], inplace=True)
df_eval['g_truth'] = df_eval['user_id'].map(g_truth)
df_eval.head()

,user_id,movie_id,rating,score,rank,g_truth
815,1,301,0,0.710759,1.0,74
815,1,435,0,0.562791,2.0,74
815,1,74,1,0.338997,3.0,74
816,2,281,1,0.581515,1.0,281
816,2,20,0,0.567110,2.0,281


In [ ]:
# hit ratio
def cal_hit_ratio(df, top_k=2):
    """Hit Ratio @ top_K"""

    top_k = df[df['rank'] <= top_k].copy()
    # golden items hit in the top_K items
    test_in_top_k = top_k[top_k['movie_id'] == top_k['g_truth']]  
    return len(test_in_top_k) * 1.0 / df['user_id'].nunique()

hr_ratio = cal_hit_ratio(df_eval)
print('HR Ratio', hr_ratio)

# ndcg
def cal_ndcg(df, top_k=2):
    top_k = df[df['rank'] <= top_k].copy()
    test_in_top_k = top_k[top_k['movie_id'] == top_k['g_truth']]  

    # the rank starts from 1
    # if rank is 1, then ndcg = 1
    test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(
        lambda x: math.log(2) / math.log(1 + x)
        )
    return test_in_top_k['ndcg'].sum() * 1.0 / df['user_id'].nunique()

ndcg = cal_ndcg(df_eval)
print('NDCG', ndcg)

HR Ratio 0.6691410392364793
NDCG 0.5513360082979945


<ipython-input-101-25c599814307>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(


# 2) Alternating Least Squares 

Singular-Value Decomposition is a matrix decomposition method for reducing a matrix to its constituent parts in order to make certain subsequent matrix calculations simpler.

With the alternating least squares approach we use the same idea but iteratively alternate between optimizing U and fixing V and vice versa. It is an iterative optimization process where we for every iteration try to arrive closer and closer to a factorized representation of our original data.

<img src='https://miro.medium.com/max/1400/1*ygHEXIhg5FtkSD3UQaldgw.webp'></img>
By randomly assigning the values in U and V and using least squares iteratively we can arrive at what weights yield the best approximation of R.

In explicit data, ratings exclusively refer to customer’s preference with the numerical value showing the magnitude of preference. In case of implicit data, numerical value often refers to a frequency which might not necessarily reflect the magnitude of customer’s preference. Hence, there is a need to deduce a confidence metric that shows customer’s confidence. So instead we focus on what we know the user has consumed and the confidence we have in whether or not they like any given item. We can for example measure how often they play a song and assume a higher confidence if they’ve listened to it 500 times vs. one time.

https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe

## 2.1 Data Loading

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/ChanCheeKean/datasets/main/pandas/user_artist_play.csv")
df = df[df['play'] > 10]
df['user_id'] = df['user'].astype("category").cat.codes
df['artist_id'] = df['artist'].astype("category").cat.codes
df.head()

In [ ]:
# Contruct a sparse matrix for our users and items containing number of plays
users = list(np.sort(df['user_id'].unique()))
artists = list(np.sort(df['artist_id'].unique()))
plays = list(df['play'])
rows = df['user_id'].astype(int)
cols = df['artist_id'].astype(int)
artist_map = dict(zip(df['artist_id'], df['artist']))

sparse_array = sparse.csr_matrix(
    (plays, (rows, cols)), 
    shape=(len(users), len(artists))
)
print(f"#Users: {len(users)}, #Artists: {len(artists)}")
pd.DataFrame(sparse_array.toarray()).head()

#Users: 956, #Artists: 49363


,0,1,2,3,4,5,6,7,8,9,...,49353,49354,49355,49356,49357,49358,49359,49360,49361,49362
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,48,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 2.2 Setting Up ALS

In [ ]:
# calculating the confidence for all users and items and randomly assign the values

def implicit_als(sparse_data, alpha_val=40, iterations=10, lambda_val=0.1, features=10):
    """ 
        iteratively compute the user (x_u) and item (y_i) vectors using the following formulas:

        x_u = ((Y.T*Y + Y.T*(Cu - I) * Y) + lambda*I)^-1 * (X.T * Cu * p(u))
        y_i = ((X.T*X + X.T*(Ci - I) * X) + lambda*I)^-1 * (Y.T * Ci * p(i))
        
        Args:
            sparse_data (csr_matrix): Our sparse user-by-item matrix
            alpha_val (int): The rate to increase confidence in a preference with more interactions.
            iterations (int): frequency to alternate between fixing and updating user and item vectors
            lambda_val (float): Regularization value
            features (int): How many latent features
        
        Returns:     
            X (csr_matrix): user vectors of size users-by-features
            Y (csr_matrix): item vectors of size items-by-features
    """

    # Calculate the confidence for each value in our data
    # if only  one interaction between a user and item 
    # the confidence will be higher than that of the unknown data given the α value
    confidence = sparse_data * alpha_val
    
    # Get the size of user rows and item columns
    user_size, item_size = sparse_data.shape
    
    # We create the user vectors X of size users-by-features, the item vectors
    # Y of size items-by-features and randomly assign the values.
    X = sparse.csr_matrix(np.random.normal(size=(user_size, features))) # len(user), 10
    Y = sparse.csr_matrix(np.random.normal(size=(item_size, features))) # len(artist), 10
    
    # Precompute I and lambda * I
    X_I = sparse.eye(user_size)
    Y_I = sparse.eye(item_size)
    
    I = sparse.eye(features)
    lI = lambda_val * I

    # Start main loop. For each iteration we first compute X and then Y
    for i in range(iterations):

        # Precompute Y-transpose-Y and X-transpose-X
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)

        # Loop through all users
        for u in range(user_size):

            # Get the user row.
            u_row = confidence[u, :].toarray() 

            # Calculate the binary preference p(u)
            p_u = u_row.copy()
            p_u[p_u != 0] = 1.0

            # Calculate Cu and Cu - I
            CuI = sparse.diags(u_row, [0])
            Cu = CuI + Y_I

            # Put it all together and compute the final formula
            yT_CuI_y = Y.T.dot(CuI).dot(Y)
            yT_Cu_pu = Y.T.dot(Cu).dot(p_u.T)
            X[u] = spsolve(yTy + yT_CuI_y + lI, yT_Cu_pu)

        for i in range(item_size):

            # Get the item column and transpose it.
            i_row = confidence[:, i].T.toarray()

            # Calculate the binary preference p(i)
            p_i = i_row.copy()
            p_i[p_i != 0] = 1.0

            # Calculate Ci and Ci - I
            CiI = sparse.diags(i_row, [0])
            Ci = CiI + X_I

            # Put it all together and compute the final formula
            xT_CiI_x = X.T.dot(CiI).dot(X)
            xT_Ci_pi = X.T.dot(Ci).dot(p_i.T)
            Y[i] = spsolve(xTx + xT_CiI_x + lI, xT_Ci_pi)

    return X, Y

user_vecs, item_vecs = implicit_als(
    sparse_array, alpha_val=40, iterations=10, lambda_val=0.1, features=10
)

In [ ]:
# more optimized code
def nonzeros(m, row):
    for index in range(m.indptr[row], m.indptr[row+1]):
        yield m.indices[index], m.data[index]
  
def least_squares_cg(Cui, X, Y, lambda_val, cg_steps=3):
    users, features = X.shape
    YtY = Y.T.dot(Y) + lambda_val * np.eye(features)

    for u in range(users):
        x = X[u]
        r = -YtY.dot(x)

        for i, confidence in nonzeros(Cui, u):
            r += (confidence - (confidence - 1) * Y[i].dot(x)) * Y[i]

        p = r.copy()
        rsold = r.dot(r)

        for it in range(cg_steps):
            Ap = YtY.dot(p)
            for i, confidence in nonzeros(Cui, u):
                Ap += (confidence - 1) * Y[i].dot(p) * Y[i]

            alpha = rsold / p.dot(Ap)
            x += alpha * p
            r -= alpha * Ap
            rsnew = r.dot(r)
            p = r + (rsnew / rsold) * p
            rsold = rsnew
        X[u] = x

def implicit_als_cg(Cui, features=20, iterations=20, lambda_val=0.1):
    user_size, item_size = Cui.shape

    X = np.random.rand(user_size, features) * 0.01
    Y = np.random.rand(item_size, features) * 0.01

    Cui, Ciu = Cui.tocsr(), Cui.T.tocsr()

    for iteration in range(iterations):
        least_squares_cg(Cui, X, Y, lambda_val)
        least_squares_cg(Ciu, Y, X, lambda_val)
    return sparse.csr_matrix(X), sparse.csr_matrix(Y)

alpha_val = 15
conf_data = (sparse_array * alpha_val).astype('double')
user_vecs, item_vecs = implicit_als_cg(conf_data, iterations=50, features=20)

## 2.3 User and Item Recommendation

In [ ]:
# find similar item/artist
item_id = 7305
item_vec = item_vecs[item_id].T
scores = item_vecs.dot(item_vec).toarray().reshape(1, -1)[0]
top_10 = np.argsort(scores)[::-1][:10]

print('Target Artist: ', artist_map[item_id])
print('\nRecomended Artist:', [artist_map[n] for n in top_10])

Target Artist:  Cathy Rivers

Recomended Artist: ['Noel Gallagher', 'Lupen Crook', 'Kyle Gabler', 'Soulsavers', '8Mm', 'Graham Colton Band', 'Duels', 'Yourcodenameis:Milo', 'Superego', 'Rooster']


In [ ]:
# recommend item for user
user_id = 100
user_interactions = sparse_array[user_id, :].toarray()

# don't recommend items the user has consumed. Set them all to 0 and the unknowns to 1
user_interactions = user_interactions.reshape(-1) + 1 
user_interactions[user_interactions > 1] = 0

# calculate recommendation
rec_vector = user_vecs[user_id, :].dot(item_vecs.T).toarray()

# scale our scores between 0 and 1 to make it all easier to interpret
min_max = MinMaxScaler()
rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1, 1))[:, 0]
recommend_vector = user_interactions * rec_vector_scaled
item_idx = np.argsort(recommend_vector)[::-1][:10]

print('Recomended Artist:', [artist_map[n] for n in item_idx])

Recomended Artist: ['Sean Lennon', 'The Bees', 'Tindersticks', 'Brian Eno & David Byrne', 'Angus And Julia Stone', 'Sea Wolf', 'Boredoms', 'Herman Düne', 'Philip Glass', 'The Frames']


## 2.4 Evaluation

In [ ]:
# 1. Masked Test Set
# take test set (some transactions not showing up in train set)
# generate predicitons, masked all predictions if it's 1 in train set
# get top N recommendations and compare with train set

# 2. Leave One out, and N negative. Refer to BPR

# 3) Bayesian Personalized Ranking (BPR)

Sample data into the triplets: a user, a positive item, a negative item:

<img src="https://miro.medium.com/max/750/1*x9UcvitvFo5IgIs-NK0FTg.webp">


https://medium.com/heyjobs-tech/building-recommendation-system-based-bayesian-personalized-ranking-using-tensorflow-2-1-b814d2704130

## 3.1 Data Loading

In [ ]:
colnames = ['user_id', 'movie_id', 'rating', 'time']
df_train = pd.read_csv('ml-100k/u1.base', delimiter = '\t', header=None, names=colnames)
df_test = pd.read_csv('ml-100k/u1.test', delimiter = '\t', header=None, names=colnames)
df_full = pd.concat([df_train, df_test])
unique_movie_ids = list(df_full['movie_id'].unique())
df_full.head()

,user_id,movie_id,rating,time
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [ ]:
### test data with rating equal 5
df_test['user_rate_freq'] = df_test.groupby('user_id')['movie_id'].transform('count')
df_test = df_test.query('(rating >= 5) & (user_rate_freq >= 20)')
ground_truth_test = df_test.groupby('user_id')['movie_id'].agg(list).reset_index()
df_test.head()

,user_id,movie_id,rating,time,user_rate_freq
0,1,6,5,887431973,137
2,1,12,5,878542960,137
3,1,14,5,874965706,137
13,1,44,5,878543541,137
20,1,60,5,875072370,137


In [ ]:
### prepare triplet
df_positive = df_full.query('rating >= 4')[['user_id', 'movie_id']]\
    .rename(columns={'movie_id' : 'positive'})

df_negative = df_full.query('rating <= 2')[['user_id', 'movie_id']]\
    .rename(columns={'movie_id' : 'negative'})

df_positive['seq'] = df_positive.groupby('user_id').cumcount()
df_negative['seq'] = df_negative.groupby('user_id').cumcount()

df_triplet = df_positive.merge(df_negative, on=['user_id', 'seq']).drop(columns=['seq'])
df_triplet.head()

,user_id,positive,negative
0,1,1,8
1,1,3,11
2,1,7,21
3,1,9,29
4,1,13,34


In [ ]:
### dataloader
class UserItemRatingDataset(Dataset):    
    def __init__(self, user, positive, negative):
        self.user_tensor = torch.LongTensor(user)
        self.positive_tensor = torch.LongTensor(positive)
        self.negative_tensor = torch.LongTensor(negative)

    def __getitem__(self, index):
        return (
            self.user_tensor[index], 
            self.positive_tensor[index], 
            self.negative_tensor[index]
        )

    def __len__(self):
        return self.user_tensor.size(0)
    
train_dataset = UserItemRatingDataset(
    user=df_triplet['user_id'].values.tolist(),
    positive=df_triplet['positive'].values.tolist(),
    negative=df_triplet['negative'].values.tolist()
    )

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
user_sample, positive_sample, negativ_sample = next(iter(train_loader))
user_sample.size()

torch.Size([256])

## 3.2 Model

In [ ]:
class BPR(nn.Module):
    
    def __init__(self, user_num, item_num, embed_num, weight_decay=0.1):
        super(BPR, self).__init__()

        # self.embed_user = torch.nn.Embedding(user_num, embed_num)
        self.embed_user = nn.Parameter(torch.empty(user_num, embed_num))

        # self.embed_item = torch.nn.Embedding(item_num, embed_num)
        self.embed_item = nn.Parameter(torch.empty(item_num, embed_num))

        torch.nn.init.xavier_normal_(self.embed_user.data)
        torch.nn.init.xavier_normal_(self.embed_item.data)
        self.weight_decay = weight_decay

    def forward(self, user, item_i, item_j):

        # user = self.embed_user(user)
        # item_i = self.embed_item(item_i)
        # item_j = self.embed_item(item_j)

        user = self.embed_user[user, :]
        item_i = self.embed_item[item_i, :]
        item_j = self.embed_item[item_j, :]

        x_ui = torch.mul(user, item_i).sum(dim=1)
        x_uj = torch.mul(user, item_j).sum(dim=1)

        x_uij = x_ui - x_uj
        log_prob = F.logsigmoid(x_uij).sum()
        regularization = self.weight_decay * (
            user.norm(dim=1).pow(2).sum() + 
            item_i.norm(dim=1).pow(2).sum() + 
            item_j.norm(dim=1).pow(2).sum()
            )
        return -log_prob + regularization

    def predict(self, u):
        """Return recommended item list given users.
        Args:
            u(torch.LongTensor): tensor stored user indexes. [batch_size,]
        Returns:
            pred(torch.LongTensor): recommended item list sorted by preference. [batch_size, item_size]
        """
        u = self.embed_user[u, :]
        scores = torch.mm(u, self.embed_item.t())
        pred = torch.argsort(scores, dim=1)
        return pred, scores

### testing
bpr_model = BPR(
    df_full['user_id'].nunique() + 1,
    df_full['movie_id'].nunique() + 1,
    128
)
optimizer = torch.optim.Adam(bpr_model.parameters(), lr=1e-3)
loss = bpr_model(user_sample, positive_sample, negativ_sample)
print('BPR Loss: ', loss.item())
# loss.backward()
# optimizer.step()

# recomend sorted by preference
pred, scores = bpr_model.predict(torch.LongTensor(df_test['user_id'].sample(10).tolist()))
print('Recommended Pred Size: ', pred.size())

BPR Loss:  190.6381072998047
Recommended Pred Size:  torch.Size([10, 1683])


## 3.3 Evaluation

If our model were to recommend 10 movies to our user, what’s the average number of movies our user would like, out of those top 10?

In [ ]:
def mean_precision_k(
    model,
    ground_truth, 
    items, 
    k
    ):
    '''
    Compare the top K recommendation with ground truth (list of items)
    '''

    scores_list = []
    for user, actual in ground_truth.items():
        pred, pred_scores = model.predict(torch.LongTensor([user]))
        # {item1: score1, item2: score2}
        predictions = dict(zip(items, pred_scores.detach().tolist()[0]))
        # length = k
        predictions = sorted(predictions.items(), key=lambda kv: kv[1], reverse=True)[:k]
        # [item10, item20, ...]
        predictions = list(OrderedDict(predictions).keys())

        score = 0.0
        num_hits = 0.0

        for i, p in enumerate(predictions):
            if p in actual:
                num_hits += 1.0
                score += num_hits / (i + 1.0)

        score = score / min(len(actual), k)
        scores_list.append(score)

    return np.mean(scores_list)

metric = mean_precision_k(
    bpr_model,
    dict(zip(ground_truth_test['user_id'], ground_truth_test['movie_id'])),
    unique_movie_ids,
    k=10
)
print(metric)

0.005130811018215599


# 4) Boltzman Machine (Bias within Sytem)

* Boltzmann Machine is a generative unsupervised model, which involves learning a probability distribution from an original dataset and using it to make inferences about never before seen data.

* Boltzmann Machine doesn’t expect input data, it generates data. Neurons generate information regardless they are hidden or visible.


https://medium.com/machine-learning-researcher/boltzmann-machine-c2ce76d94da5#:~:text=Boltzmann%20Machine%20consists%20of%20a,Machine%20is%20trying%20to%20minimize.

In [ ]:
colnames = ['user_id', 'movie_id', 'rating', 'time']
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t', header=None, names=colnames)
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t', header=None, names=colnames)
full_set = pd.concat([training_set, test_set])
full_set['time'] = pd.to_datetime(full_set['time'], unit='s')
full_set.head()

,user_id,movie_id,rating,time
0,1,1,5,1997-09-22 22:02:38
1,1,2,3,1997-10-15 05:26:11
2,1,3,4,1997-11-03 07:42:40
3,1,4,3,1997-10-15 05:25:19
4,1,5,3,1998-03-13 01:15:12


In [ ]:
nb_users = full_set['user_id'].nunique()
nb_movies = full_set['movie_id'].nunique()
training_set['rating'] = training_set['rating'].apply(lambda x: -1 if x < 1 else(0 if x < 3 else 1))
test_set['rating'] = test_set['rating'].apply(lambda x: -1 if x < 1 else(0 if x < 3 else 1))
training_set = training_set.groupby(['user_id', 'movie_id'])['rating'].max().unstack().fillna(-1)
test_set = test_set.groupby(['user_id', 'movie_id'])['rating'].max().unstack().fillna(-1)
test_set = test_set.reindex(columns = training_set.columns.tolist()).fillna(-1)

print(training_set.shape)
print(test_set.shape)
training_set.head()

(943, 1650)
(459, 1650)


movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,0.0,1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [ ]:
training_set = np.array(training_set, dtype='int')
test_set = np.array(test_set, dtype='int')
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

print(training_set.size(), test_set.size())

torch.Size([943, 1650]) torch.Size([459, 1650])


In [ ]:
# boltman machine class
class RBM():
    '''
    Parameters:
    nv: int. Number of Visible Nodes, supposed to be the size of data
    nh: int. Number of Hidden Nodes
    '''

    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)

    def sample_h(self, x):
        '''
        To get probability of hidden node given visible node
        Parameters:
        x: int. Vector of neurons
        '''    
        # [m, item_len] x [item_len, m] --> [m, m]
        wx = torch.mm(x, self.W.t())

        # [1, m] --> [m, m]
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)

        # Draws binary random numbers from Bernoulli distribution
        # [m, m], [m, m]
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_v(self, y):
        # [m, m] x [m, item_len] --> [m, item_len]
        wy = torch.mm(y, self.W)

        # [m, item_len]
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)

    def train(self, v0, vk, ph0, phk):
        # contrast divergence algorithm
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

batch_size = 100
rbm = RBM(training_set.size()[1], 100)

### testing
batch_size = 100
vk = training_set[: batch_size]
v0 = training_set[: batch_size]
ph0, _ = rbm.sample_h(v0)
print("ph0 size: ", ph0.size())

for k in range(10):
    _, hk = rbm.sample_h(vk)
    _, vk = rbm.sample_v(hk)
    print("vk size: ", vk.size())
    vk[v0 < 0] = v0[v0 < 0]
    break

phk, _ = rbm.sample_h(vk)
print("phk size: ", phk.size())
rbm.train(v0, vk, ph0, phk)

train_loss = torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
print(train_loss)

ph0 size:  torch.Size([100, 100])
vk size:  torch.Size([100, 1650])
phk size:  torch.Size([100, 100])
tensor(0.4681)


In [ ]:
# model training
nb_epoch = 5
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.0
    
    # Gibb Sampling
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user : id_user + batch_size]
        v0 = training_set[id_user : id_user + batch_size]
        ph0,_ = rbm.sample_h(v0)
        # monte carlo
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        s += 1.
    print('epoch: ' + str(epoch)+' loss: ' + str(train_loss/s))

epoch: 1 loss: tensor(0.2543)
epoch: 2 loss: tensor(0.2466)
epoch: 3 loss: tensor(0.2474)
epoch: 4 loss: tensor(0.2465)
epoch: 5 loss: tensor(0.2451)


In [ ]:
### evaluation
# testing
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _, h = rbm.sample_h(v)
        _, v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: ' + str(test_loss/s))

test loss: tensor(0.2309)
